# openai api key test

## API key Loading

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# 환경변수 로딩
load_dotenv(override=True)

# 메모리에 로딩된 값을 api_key 변수에 대입
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# print(api_key)

## 라이브러리 로딩 및 객체 생성

In [2]:
from openai import OpenAI
# openai api 인증 및 OpenAI 객체 생성
client = OpenAI(api_key=OPENAI_API_KEY)

## TTS (Text to Speech)

In [3]:
with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="alloy",
    response_format="wav",
    input="오늘은 내가 가장 행복한 날입니다."
) as response:
    response.stream_to_file("speech_alloy.wav"
)

In [4]:
# gpt-4o-mini-tts

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="cedar",
    response_format="wav",
    input="안녕하세요. 반갑습니다. 오늘도 좋은 하루 보내세요."
) as response:
    response.stream_to_file("speech_cedar.wav"
)

## STT (Speech to Text)

In [4]:
# 소리 파일
audio_file = open("speech_cedar.wav", "rb")
type(audio_file)

_io.BufferedReader

In [5]:
# STT (Speech to Text)
response = client.audio.transcriptions.create(
    model = "whisper-1",
    response_format ="text",
    # response_format ="verbose_json",
    # response_format ="json",
    file = audio_file,
)

In [6]:
# dict로 변환,
#   {'text': '안녕하세요. 반갑습니다. 오늘도 좋은 하루 보내세요.',
#       'logprobs': None,
#       'usage': UsageDuration(seconds=4.0, type='duration')}
response.replace("\n","") # "\n" 제거

'안녕하세요. 반갑습니다. 오늘도 좋은 하루 보내세요.'

In [22]:
# text 포맷 응답 결과
# '안녕하세요. 반갑습니다. 오늘도 좋은 하루 보내세요.\n'
# verbose_json 포맷 응답 결과
# TranscriptionVerbose(duration=3.3499999046325684, language='korean', text='안녕하세요. 반갑습니다. 오늘도 좋은 하루 보내세요.', segments=[TranscriptionSegment(id=0, avg_logprob=-0.39627134799957275, compression_ratio=0.9102563858032227, end=3.5999999046325684, no_speech_prob=0.007054213434457779, seek=0, start=0.0, temperature=0.0, text=' 안녕하세요. 반갑습니다. 오늘도 좋은 하루 보내세요.', tokens=[50364, 19289, 13, 16396, 27358, 3115, 13, 47455, 16460, 33918, 39833, 1041, 25918, 13, 50544])], usage=Usage(seconds=4.0, type='duration'), words=None, task='transcribe')


- STT 수행할 때, 파일이 클 경우(25MB 이상) 처리 방법

In [7]:
from pydub import AudioSegment
audio_file = "datas/audio.mp3"
song = AudioSegment.from_mp3(audio_file)
song

In [8]:
from pydub import AudioSegment
import math
import openai

# 오디오 파일 불러오기
audo_file = "datas/audio.mp3"
song = AudioSegment.from_mp3(audo_file)

# Whisper는 약 25MB 제한이 있으므로, 시간 기준으로 나누기
# 파일 크기에 따라 다르지만, 보통 10분(600초) 정도면 20MB 내외
chunk_length_ms = 10 * 60 * 1000  # 10분 단위로 자르기 (단위: 밀리초)
num_chunks = math.ceil(len(song) / chunk_length_ms)

print(f"총 {num_chunks}개의 조각으로 분할합니다.")

# 전체 텍스트를 담을 리스트
full_transcript = []

for i in range(num_chunks):
    start = i * chunk_length_ms
    end = min((i + 1) * chunk_length_ms, len(song))
    chunk = song[start:end]
    chunk_filename = f"chunk_{i}.mp3"
    chunk.export(chunk_filename, format="mp3")
    
    print(f"{chunk_filename} 변환 중...")

    # Whisper API 호출
    with open(chunk_filename, "rb") as audio_file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="text",
        )
        full_transcript.append(response)
        print(f"{chunk_filename} 변환 완료.")

# 모든 조각의 텍스트를 합치기
final_text = "\n".join(full_transcript)

# 결과 저장
with open("full_transcript.txt", "w", encoding="utf-8") as f:
    f.write(final_text)

print("전체 파일의 텍스트 변환이 완료되었습니다!")

총 1개의 조각으로 분할합니다.
chunk_0.mp3 변환 중...
chunk_0.mp3 변환 완료.
전체 파일의 텍스트 변환이 완료되었습니다!


## AudioSegment.from_mp3 파일이 안읽혀질때
Windows에서 FFmpeg 설치
1. Chocolatey 설치  
- Chocolatey가 설치되어 있지 않다면 다음 단계를 따라 설치할 수 있습니다:

    1) 관리자 권한으로 PowerShell을 엽니다.

    2) 다음 명령어를 실행하여 Chocolatey를 설치합니다:  
    - https://chocolatey.org/install 에서 복사해서 설치
    
    3) Chocolatey 설치가 완료되면, 다음 명령어로 FFmpeg를 설치할 수 있습니다:  
    - choco install ffmpeg

    4) vs-code, 터미널 모두 닫았다가 다시 시작